# SimFin FAANG analysis Neural Prophet Forecasting
_By Nick Brooks, December 2021_

In [ ]:
!pip install simfin
!pip install neuralprophet

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

import time

from neuralprophet import NeuralProphet

# Import the main functionality from the SimFin Python API.
import simfin as sf
print("Simfin Version: ",sf.__version__)

sf.set_data_dir('~/simfin_data/')
sf.set_api_key(api_key='free')

sns.set_style("whitegrid")
sns.set_palette("tab10")

pd.options.display.max_colwidth = 500
pd.options.display.max_rows = 999
pd.options.display.width = 300
pd.options.display.max_columns = 100

notebookstart = time.time()

In [ ]:
df = sf.load(dataset='income', variant='annual', market='us')
df['Date'] = pd.to_datetime(df["Report Date"])
df['Year'] = df['Date'].dt.year
print("DF Shape: {} Rows, {} Columns".format(*df.shape))
print("Number of Tickers: {}".format(len(df.Ticker.unique())))

In [ ]:
df_prices = sf.load_shareprices(variant='daily', market='us').reset_index().set_index("Date")
print("DF Shape: {} Rows, {} Columns".format(*df_prices.shape))
print("Number of Tickers: {}".format(len(df_prices.Ticker.unique())))

#### FAANG Analysis

In [ ]:
allticker = ['FB',"AMZN","AAPL","NFLX","GOOG"]

In [ ]:
check_ticker = [x for x in allticker if x not in df_prices.loc[df_prices.Ticker.isin(allticker)].Ticker.unique()]
print("All Tickers Available") if not check_ticker else print("Missing:", check_ticker)

stock = df_prices.loc[df_prices.Ticker.isin(allticker)].copy()
stock["DateTime"] = pd.to_datetime(stock.index)
stock["Year"] = pd.to_datetime(stock.DateTime).dt.year
stock.set_index("DateTime", inplace=True)
display(stock.sample(3))

In [ ]:
check_ticker = [x for x in allticker if x not in df.loc[df.Ticker.isin(allticker)].Ticker.unique()]
print("All Tickers Available") if not check_ticker else print("Missing:", check_ticker)

fndm = df.loc[df.Ticker.isin(allticker)].set_index("Year")
display(fndm.sample(3))

In [ ]:
f, ax = plt.subplots(figsize=[8,4])
stock.groupby("Ticker")["Adj. Close"].plot(ax=ax)
ax.set_title("FAANG Adj. Close Price")
ax.legend()
plt.show()

### Analyze Fundamentals

In [ ]:
# ticker = allticker[0]

for ticker in allticker:
    tkr_fndm = fndm.loc[fndm.Ticker == ticker].copy()

    metrics = ["Gross Profit",'Revenue', "Operating Expenses"]
    metrics_m = []
    for m in metrics:
        new_col = m + " (Billion USD)"
        metrics_m.append(new_col)
        tkr_fndm[new_col] = np.abs(tkr_fndm[m] / 1000000000)

    tkr_fndm["Profit to Revenue Ratio"] = tkr_fndm['Gross Profit'] / tkr_fndm['Revenue']
    tkr_fndm["R&D to Revenue Ratio"] = np.abs(tkr_fndm['Research & Development']) / tkr_fndm['Revenue']

    f,ax = plt.subplots(1,2,figsize = [12,4])
    tkr_fndm[metrics_m].plot(ax=ax[0], linewidth=2)
    tkr_fndm[["Profit to Revenue Ratio","R&D to Revenue Ratio"]].plot(ax=ax[1], linewidth=2)

    ax[0].set_title("{} Operational Metrics".format(ticker))
    ax[1].set_title("{} Financial Ratios".format(ticker))
    ax[1].set_ylim(0,1)
    plt.tight_layout(pad=2)
    plt.show()

### Time Series Predictions

Experiment with Neural Prophet Package to forecast and extract components.

In [ ]:
ts = stock.loc[(stock.Ticker == "AMZN") & (stock.Year > 2016), "Adj. Close"]
ts.plot()
plt.title("Amazon Adj. Closing Stock Price")
plt.show()

In [ ]:
train = ts.reset_index().rename(columns={"DateTime":"ds", "Adj. Close":"y"})

m = NeuralProphet()
metrics = m.fit(train, freq="D")
future = m.make_future_dataframe(train, periods=120)
forecast = m.predict(future)

In [ ]:
fig_forecast = m.plot(forecast)

In [ ]:
fig_components = m.plot_components(forecast)

In [ ]:
fig_model = m.plot_parameters()

Model seems to struggle with non-stationary, non-highly seasonal data

In [ ]:
print("Script Complete - Runtime: {:.2f} Minutes".format((time.time() - notebookstart) / 60))